## Implied Volatility

The Black-Scholes pricing formula is usually used to solve for implied volatility. Quite simply, this means setting the Black-Scholes pricing formula equal to the market observed price and using a root finding algorithm to solve for the volatility parameter which sets the difference (between model and market price) to zero.
The implied volatility is generally used for making trading decisions, calibrating other more exotic securities and researching market anomolies. For example, many traders use the so called volatility skew to understand the relative value of an option to other options trading in the market.

### Import Libraries

In [11]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

### Initial Data

In [9]:
# underlying stock price
S = 45.0
# series of underlying stock prices to demonstrate a payoff profile
S_ = np.arange(35.0, 55.0, 0.01) # strike price
K = 45.0
# time to expiration
t = 164.0 / 365.0
# risk free rate
r = 0.02
# volatility
vol = 0.25

### Black-Scholes

In [3]:
def N(z):
    """ Normal cumulative density function
    :param z: point at which cumulative density is calculated 
    return: cumulative density under normal curve
    """
    return norm.cdf(z)

def black_scholes_call_value(S, K, r, t, vol): 
    """ Black-Scholes call option
    :param S: underlying
    :param K: strike price
    :param r: rate
    :param t: time to expiration
    :param vol: volatility
    :return: BS call option value
    """
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t) 
    d2 = d1 - (vol * np.sqrt(t))
    
    return N(d1) * S - N(d2) * K * np.exp(-r * t) 

def black_scholes_put_value(S, K, r, t, vol):
    """ Black-Scholes put option
    :param S: underlying
    :param K: strike price
    :param r: rate
    :param t: time to expiration
    :param vol: volatility
    :return: BS put option value
    """
    d1 = (1.0 / (vol * np.sqrt(t))) * (np.log(S / K) + (r + 0.5 * vol ** 2.0) * t) 
    d2 = d1 - (vol * np.sqrt(t))
    
    return N(-d2) * K * np.exp(-r * t) - N(-d1) * S

### Implied Volatility for a Call Option

In [4]:
def call_implied_volatility_objective_function( S, K, r, t, vol, call_option_market_price):
    return call_option_market_price - black_scholes_call_value(S, K, r, t, vol)

In [5]:
def call_implied_volatility(S, K, r, t, call_option_market_price, a=-2.0, b=2.0, xtol=1e-6):
    """ Call implied volatility function
    :param a: lower bound for brentq method
    :param b: upper gound for brentq method
    :param xtol: tolerance which is considered good enough
    :return: volatility to sets the difference between market and model price to zero
    """
    # avoid mirroring outer scope
    _S, _K, _r, _t, _call_option_market_price = S, K, r, t, call_option_market_price # define a nested function that takes our target param as the input
    
    def fcn(vol):
        # returns the difference between market and model price at given volatility
        return call_implied_volatility_objective_function( _S, _K, _r, _t, vol, _call_option_market_price)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(fcn, a=a, b=b, xtol=xtol)
        # if the results are *too* small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    # if it fails then we return np.nan so we can later interpolate the results
    except ValueError: 
        return np.nan

### Implied Volatility for a Put Option

In [6]:
def put_implied_volatility_objective_function(S, K, r, t, vol, put_option_market_price):
    return put_option_market_price - black_scholes_put_value(S, K, r, t, vol)

In [7]:
def put_implied_volatility(S, K, r, t, put_option_market_price, a=-2.0, b=2.0, xtol=1e-6):
    """ Put implied volatility function
    :param a: lower bound for brentq method
    :param b: upper gound for brentq method
    :param xtol: tolerance which is considered good enough
    :return: volatility to sets the difference between market and model price to zero
    """
    # avoid mirroring out scope
    _S, _K, _r, _t, _put_option_market_price = S, K, r, t, put_option_market_price # define a nsted function that takes our target param as the input
    
    def fcn(vol):
        # returns the difference between market and model price at given volatility
        return put_implied_volatility_objective_function( _S, _K, _r, _t, vol, _put_option_market_price)
    
    # first we try to return the results from the brentq algorithm
    try:
        result = brentq(fcn, a=a, b=b, xtol=xtol)
        # if the results are *too* small, sent to np.nan so we can later interpolate
        return np.nan if result <= 1.0e-6 else result
    
    # if it fails then we return np.nan so we can later interpolate the results
    except ValueError: 
        return np.nan

### Implied Vol

In [12]:
# get the call and put values to test the implied volatility output
call_model_price = black_scholes_call_value(S, K, r, t, vol) 
print(
    "Call implied volatility if market and model were equal (should be close to 0.25) %0.6f"
% call_implied_volatility(S, K, r, t, call_model_price) )

put_model_price = black_scholes_put_value(S, K, r, t, vol) 
print(
    "Put implied volatility if market and model were equal (should be close to 0.25) %0.6f"
% put_implied_volatility(S, K, r, t, put_model_price) )

Call implied volatility if market and model were equal (should be close to 0.25) 0.250000
Put implied volatility if market and model were equal (should be close to 0.25) 0.250000
